In [7]:
# install requirements (one-time)
#!pip install google-cloud-bigquery google-cloud-storage tqdm
!pip install pip install google-auth

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


## Read Configurations

## Define Imports and Functions

In [19]:
import os
import random
import string
import time
from typing import List, Optional, Tuple


from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import storage
from google.api_core.exceptions import NotFound, Forbidden, GoogleAPICallError
from tqdm import tqdm

In [20]:
# Creates new name with 2 number suffix
def _make_random_suffix(n=2):
    return ''.join(random.choices('0123456789', k=n))

In [21]:
def _get_source_client_from_key(key_path: str) -> bigquery.Client:
    creds = service_account.Credentials.from_service_account_file(key_path)
    return bigquery.Client(credentials=creds, project=creds.project_id)

In [22]:
src_creds = service_account.Credentials.from_service_account_file("../secrets/darry-r26.json")

In [39]:
src_client = bigquery.Client(credentials=src_creds, project=src_creds.project_id)

In [35]:
dataset_names = []
for datasets in src_client.list_datasets():
    # save under a plan
    dataset_names.append(f'{datasets.project}.{datasets.dataset_id}')

In [37]:
len(dataset_names)

34

In [40]:
for projects in src_client.list_projects():
    print(projects.project_id)

data-analytics-ns


In [43]:
# for dataset in dataset_names:
for table in src_client.list_tables('data-analytics-ns.course15'):
    print(table.full_table_id)

data-analytics-ns:course15.cc_funnel
data-analytics-ns:course15.cc_funnel_kpi
data-analytics-ns:course15.cc_funnel_local
data-analytics-ns:course15.cc_parcel
data-analytics-ns:course15.cc_parcel_kpi
data-analytics-ns:course15.cc_parcel_kpi_formatted
data-analytics-ns:course15.cc_parcel_kpi_global
data-analytics-ns:course15.cc_parcel_kpi_month
data-analytics-ns:course15.cc_parcel_kpi_priority
data-analytics-ns:course15.cc_parcel_kpi_priority_by_ratio
data-analytics-ns:course15.cc_parcel_kpi_transporter
data-analytics-ns:course15.cc_parcel_product
data-analytics-ns:course15.circle_sales
data-analytics-ns:course15.circle_sales_daily
data-analytics-ns:course15.circle_stock
data-analytics-ns:course15.circle_stock_cat
data-analytics-ns:course15.circle_stock_copy
data-analytics-ns:course15.circle_stock_d7_ch2
data-analytics-ns:course15.circle_stock_kpi
data-analytics-ns:course15.circle_stock_kpi_top
data-analytics-ns:course15.circle_stock_model_type
data-analytics-ns:course15.circle_stock_nam

In [44]:
src_client.query_and_wait('select * from data-analytics-ns.course15.cc_parcel_kpi_global')